In [33]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, concatenate, GRU
from tensorflow.keras.optimizers import RMSprop
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [2]:
pd.set_option('display.max_columns', None) 
df = pd.read_csv('data/pitcher_main_trimmed.csv')
df = df.drop(['player_name', 'game_date'], axis=1)
df

,pitch_type,release_speed,release_pos_x,release_pos_z,pitcher,events,description,zone,p_throws,type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,inning,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,pitch_number,pitch_name,delta_run_exp
0,ff,90.3,1.68,6.60,115399,strikeout,swinging_strike,13.0,l,s,1,2,2008,0.40,0.94,-0.73,1.28,6,-7.021865,-132.301574,-8.362541,6.289000,34.467000,-19.624000,3.57,1.75,4,4-seam fastball,-0.075
1,si,85.0,2.10,6.08,115399,ordinary,ball,14.0,l,b,0,2,2008,1.41,0.17,2.10,1.77,6,-3.316781,-124.853944,-2.798613,14.684000,32.913000,-29.805000,3.57,1.75,3,sinker,0.010
2,si,83.7,1.96,6.36,115399,ordinary,swinging_strike,14.0,l,s,0,1,2008,1.23,0.68,1.31,2.04,6,-4.352248,-122.837225,-3.718380,12.749000,31.332000,-24.795000,3.57,1.75,2,sinker,-0.023
3,ff,91.5,1.68,6.62,115399,ordinary,called_strike,14.0,l,s,0,0,2008,0.70,1.39,0.93,1.80,6,-3.669918,-134.169651,-8.573571,8.963000,34.367000,-14.114000,3.57,1.75,1,4-seam fastball,-0.016
4,ff,91.5,1.52,6.54,115399,field_out,hit_into_play,4.0,l,x,0,2,2008,0.56,1.23,-0.66,2.40,6,-6.935053,-134.094163,-6.479771,8.162000,33.240000,-16.476000,3.38,1.50,3,4-seam fastball,-0.113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9544338,si,96.2,-1.66,5.70,666168,ordinary,foul,4.0,r,s,0,0,2022,-1.79,0.64,-0.41,2.54,9,7.428355,-139.888778,-4.260597,-25.232534,31.096104,-22.963414,3.49,1.60,1,sinker,-0.034
9544339,ff,92.5,-2.49,5.99,608723,ordinary,ball,11.0,r,b,0,0,2022,-0.73,1.46,-0.36,3.99,6,6.965142,-134.626401,-2.524665,-10.246435,26.104120,-13.900579,3.26,1.51,1,4-seam fastball,0.033
9544340,si,91.7,2.12,6.44,571578,ordinary,called_strike,8.0,l,s,0,0,2022,1.16,1.25,-0.02,1.63,1,-7.967493,-133.046151,-8.987455,15.537407,30.338976,-15.552218,3.39,1.55,1,sinker,-0.034
9544341,cu,81.9,-1.63,6.31,641907,ordinary,ball,14.0,r,b,0,0,2022,0.94,-1.16,0.66,1.30,8,3.347026,-119.221540,-2.480205,8.186566,24.508845,-42.874474,3.59,1.53,1,curveball,0.033


In [3]:
X = df.drop('pitcher', axis=1)
y = df['pitcher']

In [4]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.6, random_state=42)

In [5]:
X_train_encoded = pd.get_dummies(X_train)
X_train_encoded

,release_speed,release_pos_x,release_pos_z,zone,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,inning,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,pitch_number,delta_run_exp,pitch_type_ab,pitch_type_ch,pitch_type_cs,pitch_type_cu,pitch_type_ep,pitch_type_fa,pitch_type_fc,pitch_type_ff,pitch_type_fo,pitch_type_fs,pitch_type_kc,pitch_type_kn,pitch_type_po,pitch_type_sc,pitch_type_si,pitch_type_sl,pitch_type_st,pitch_type_sv,events_double,events_double_play,events_field_out,events_force_out,events_grounded_into_double_play,events_hit_by_pitch,events_home_run,events_ordinary,events_sac_bunt,events_sac_fly,events_single,events_strikeout,events_triple,events_walk,description_ball,description_blocked_ball,description_bunt_foul_tip,description_called_strike,description_foul,description_foul_bunt,description_foul_pitchout,description_foul_tip,description_hit_by_pitch,description_hit_into_play,description_missed_bunt,description_swinging_strike,description_swinging_strike_blocked,p_throws_l,p_throws_r,type_b,type_s,type_x,pitch_name_4-seam fastball,pitch_name_automatic ball,pitch_name_changeup,pitch_name_curveball,pitch_name_cutter,pitch_name_eephus,pitch_name_forkball,pitch_name_knuckle curve,pitch_name_knuckleball,pitch_name_other,pitch_name_pitch out,pitch_name_screwball,pitch_name_sinker,pitch_name_slider,pitch_name_slow curve,pitch_name_slurve,pitch_name_split-finger,pitch_name_sweeper
7050222,83.6,-0.76,5.93,14.0,0,2,2018,-0.11,0.11,0.36,1.23,7,2.743919,-121.654828,-4.644026,-1.521414,19.768491,-30.589423,3.48,1.55,3,0.014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
7125903,92.1,-1.52,5.34,6.0,1,1,2018,-1.16,0.14,0.42,2.61,7,7.190442,-133.825435,-1.405367,-14.929541,26.643651,-30.458410,3.48,1.78,3,-0.056,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4728592,93.4,-2.44,6.13,2.0,0,1,2015,-0.11,0.94,-0.12,2.72,2,6.262852,-136.755256,-4.447418,-2.510000,25.910000,-19.770000,3.15,1.32,2,-0.130,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
389108,82.8,-1.56,6.85,14.0,0,1,2008,0.55,0.96,0.72,1.01,6,3.956691,-121.068538,-7.805802,4.512000,23.230000,-21.532000,3.72,1.67,2,-0.023,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6431105,91.5,-1.48,6.17,6.0,1,0,2017,0.56,1.43,0.75,2.55,4,4.427583,-132.904390,-6.381064,5.768874,27.698199,-13.886755,3.35,1.42,2,-0.124,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9524682,87.2,-1.86,5.86,11.0,0,2,2022,-0.52,1.64,-1.39,4.43,5,2.201373,-127.078835,-0.641876,-6.076769,26.577758,-14.526367,3.43,1.59,3,0.042,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2234489,88.5,-2.00,6.43,11.0,0,1,2011,-0.24,1.68,-0.27,4.31,8,4.838106,-129.790344,-2.092722,-3.653000,26.508000,-13.372000,3.84,2.02,2,0.047,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4304572,87.1,-1.55,5.72,7.0,2,2,2014,-0.21,0.07,-0.30,1.73,3,3.502885,-127.737765,-2.498979,-2.882000,25.310000,-30.918000,3.42,1.62,7,-0.201,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
6550634,82.2,1.89,5.

In [6]:
train_cols = X_train_encoded.columns

In [7]:
X_val_encoded = pd.get_dummies(X_val)
X_val_encoded = X_val_encoded.reindex(columns = train_cols, fill_value = 0)

In [8]:
X_test_encoded = pd.get_dummies(X_test)
X_test_encoded = X_test_encoded.reindex(columns = train_cols, fill_value = 0)

In [9]:
scaler = StandardScaler().fit(X_train_encoded)

In [10]:
X_train_encoded

,release_speed,release_pos_x,release_pos_z,zone,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,inning,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,pitch_number,delta_run_exp,pitch_type_ab,pitch_type_ch,pitch_type_cs,pitch_type_cu,pitch_type_ep,pitch_type_fa,pitch_type_fc,pitch_type_ff,pitch_type_fo,pitch_type_fs,pitch_type_kc,pitch_type_kn,pitch_type_po,pitch_type_sc,pitch_type_si,pitch_type_sl,pitch_type_st,pitch_type_sv,events_double,events_double_play,events_field_out,events_force_out,events_grounded_into_double_play,events_hit_by_pitch,events_home_run,events_ordinary,events_sac_bunt,events_sac_fly,events_single,events_strikeout,events_triple,events_walk,description_ball,description_blocked_ball,description_bunt_foul_tip,description_called_strike,description_foul,description_foul_bunt,description_foul_pitchout,description_foul_tip,description_hit_by_pitch,description_hit_into_play,description_missed_bunt,description_swinging_strike,description_swinging_strike_blocked,p_throws_l,p_throws_r,type_b,type_s,type_x,pitch_name_4-seam fastball,pitch_name_automatic ball,pitch_name_changeup,pitch_name_curveball,pitch_name_cutter,pitch_name_eephus,pitch_name_forkball,pitch_name_knuckle curve,pitch_name_knuckleball,pitch_name_other,pitch_name_pitch out,pitch_name_screwball,pitch_name_sinker,pitch_name_slider,pitch_name_slow curve,pitch_name_slurve,pitch_name_split-finger,pitch_name_sweeper
7050222,83.6,-0.76,5.93,14.0,0,2,2018,-0.11,0.11,0.36,1.23,7,2.743919,-121.654828,-4.644026,-1.521414,19.768491,-30.589423,3.48,1.55,3,0.014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
7125903,92.1,-1.52,5.34,6.0,1,1,2018,-1.16,0.14,0.42,2.61,7,7.190442,-133.825435,-1.405367,-14.929541,26.643651,-30.458410,3.48,1.78,3,-0.056,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4728592,93.4,-2.44,6.13,2.0,0,1,2015,-0.11,0.94,-0.12,2.72,2,6.262852,-136.755256,-4.447418,-2.510000,25.910000,-19.770000,3.15,1.32,2,-0.130,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
389108,82.8,-1.56,6.85,14.0,0,1,2008,0.55,0.96,0.72,1.01,6,3.956691,-121.068538,-7.805802,4.512000,23.230000,-21.532000,3.72,1.67,2,-0.023,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6431105,91.5,-1.48,6.17,6.0,1,0,2017,0.56,1.43,0.75,2.55,4,4.427583,-132.904390,-6.381064,5.768874,27.698199,-13.886755,3.35,1.42,2,-0.124,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9524682,87.2,-1.86,5.86,11.0,0,2,2022,-0.52,1.64,-1.39,4.43,5,2.201373,-127.078835,-0.641876,-6.076769,26.577758,-14.526367,3.43,1.59,3,0.042,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2234489,88.5,-2.00,6.43,11.0,0,1,2011,-0.24,1.68,-0.27,4.31,8,4.838106,-129.790344,-2.092722,-3.653000,26.508000,-13.372000,3.84,2.02,2,0.047,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4304572,87.1,-1.55,5.72,7.0,2,2,2014,-0.21,0.07,-0.30,1.73,3,3.502885,-127.737765,-2.498979,-2.882000,25.310000,-30.918000,3.42,1.62,7,-0.201,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
6550634,82.2,1.89,5.

In [11]:
X_train_encoded_scaled = pd.DataFrame(scaler.transform(X_train_encoded), columns=train_cols)
X_val_encoded_scaled = pd.DataFrame(scaler.transform(X_val_encoded), columns=train_cols)
X_test_encoded_scaled = pd.DataFrame(scaler.transform(X_test_encoded), columns=train_cols)

In [12]:
print(X_train_encoded_scaled.columns)

Index(['release_speed', 'release_pos_x', 'release_pos_z', 'zone', 'balls',
       'strikes', 'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z',
       'inning', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'pitch_number', 'delta_run_exp', 'pitch_type_ab', 'pitch_type_ch',
       'pitch_type_cs', 'pitch_type_cu', 'pitch_type_ep', 'pitch_type_fa',
       'pitch_type_fc', 'pitch_type_ff', 'pitch_type_fo', 'pitch_type_fs',
       'pitch_type_kc', 'pitch_type_kn', 'pitch_type_po', 'pitch_type_sc',
       'pitch_type_si', 'pitch_type_sl', 'pitch_type_st', 'pitch_type_sv',
       'events_double', 'events_double_play', 'events_field_out',
       'events_force_out', 'events_grounded_into_double_play',
       'events_hit_by_pitch', 'events_home_run', 'events_ordinary',
       'events_sac_bunt', 'events_sac_fly', 'events_single',
       'events_strikeout', 'events_triple', 'events_walk', 'description_ball',
       'description_blocked_ball', 'description_bunt_foul_tip',
   

In [13]:
del X_train, X_val, X_test, X_train_encoded, X_val_encoded, X_test_encoded, df, X_temp, y_temp

In [14]:
pitch_name_inputs = Input(shape=(X_train_encoded_scaled[['pitch_name_4-seam fastball', 'pitch_name_automatic ball', 'pitch_name_changeup', 'pitch_name_curveball', 'pitch_name_cutter', 'pitch_name_eephus', 'pitch_name_forkball', 'pitch_name_knuckle curve', 'pitch_name_knuckleball', 'pitch_name_other', 'pitch_name_pitch out', 'pitch_name_screwball', 'pitch_name_sinker', 'pitch_name_slider', 'pitch_name_slow curve', 'pitch_name_slurve', 'pitch_name_split-finger', 'pitch_name_sweeper']].shape[1],))

In [15]:
events_inputs = Input(shape=(X_train_encoded_scaled[['events_double', 'events_double_play', 'events_field_out', 'events_force_out', 'events_grounded_into_double_play', 'events_hit_by_pitch', 'events_home_run', 'events_ordinary', 'events_sac_bunt', 'events_sac_fly', 'events_single', 'events_strikeout', 'events_triple', 'events_walk']].shape[1],))

In [16]:
description_inputs = Input(shape=(X_train_encoded_scaled[['description_ball', 'description_blocked_ball', 'description_bunt_foul_tip', 'description_called_strike', 'description_foul', 'description_foul_bunt', 'description_foul_pitchout', 'description_foul_tip', 'description_hit_by_pitch', 'description_hit_into_play', 'description_missed_bunt', 'description_swinging_strike', 'description_swinging_strike_blocked']].shape[1],))

In [17]:
pitch_type_inputs = Input(shape=(X_train_encoded_scaled[['pitch_type_ab', 'pitch_type_ch', 'pitch_type_cs', 'pitch_type_cu', 'pitch_type_ep', 'pitch_type_fa', 'pitch_type_fc', 'pitch_type_ff', 'pitch_type_fo', 'pitch_type_fs', 'pitch_type_kc', 'pitch_type_kn', 'pitch_type_po', 'pitch_type_sc', 'pitch_type_si', 'pitch_type_sl', 'pitch_type_st', 'pitch_type_sv']].shape[1],))

In [18]:
velocity_inputs = Input(shape=(X_train_encoded_scaled[['vx0', 'vy0', 'vz0']].shape[1],))

In [19]:
pfx_inputs = Input(shape=(X_train_encoded_scaled[['pfx_x', 'pfx_z']].shape[1],))

In [20]:
plate_inputs = Input(shape=(X_train_encoded_scaled[['plate_x', 'plate_z']].shape[1],))

In [21]:
release_pos_inputs = Input(shape=(X_train_encoded_scaled[['release_pos_x', 'release_pos_z']].shape[1],))

In [22]:
acceleration_inputs = Input(shape=(X_train_encoded_scaled[['ax', 'ay', 'az']].shape[1],))

In [23]:
sz_inputs = Input(shape=(X_train_encoded_scaled[['sz_top', 'sz_bot']].shape[1],))

In [24]:
p_throws_inputs = Input(shape=(X_train_encoded_scaled[['p_throws_l', 'p_throws_r']].shape[1],))

In [25]:
type_inputs = Input(shape=(X_train_encoded_scaled[['type_b', 'type_s', 'type_x']].shape[1],))

In [26]:
pitch_name_dense = Dense(32, activation='relu')(pitch_name_inputs)
events_dense = Dense(32, activation='relu')(events_inputs)
description_dense = Dense(32, activation='relu')(description_inputs)
pitch_type_dense = Dense(32, activation='relu')(pitch_type_inputs)
velocity_dense = Dense(32, activation='relu')(velocity_inputs)
pfx_dense = Dense(32, activation='relu')(pfx_inputs)
plate_dense = Dense(32, activation='relu')(plate_inputs)
release_pos_dense = Dense(32, activation='relu')(release_pos_inputs)
acceleration_dense = Dense(32, activation='relu')(acceleration_inputs)
sz_dense = Dense(32, activation='relu')(sz_inputs)
p_throws_dense = Dense(32, activation='relu')(p_throws_inputs)
type_dense = Dense(32, activation='relu')(type_inputs)

In [29]:
concat = concatenate([pitch_name_dense, events_dense, description_dense, pitch_type_dense, velocity_dense, pfx_dense, plate_dense, release_pos_dense, acceleration_dense, sz_dense, p_throws_dense, type_dense])

In [30]:
dense1 = Dense(128, activation='relu')(concat)
dropout = Dropout(0.2)(dense1)
dense2 = Dense(64, activation='relu')(dropout)

In [31]:
output = Dense(1, activation='linear')(dense2)  # change based on your needs

In [34]:
model = Model(inputs=[pitch_name_inputs, events_inputs, description_inputs, pitch_type_inputs, velocity_inputs, pfx_inputs, plate_inputs, release_pos_inputs, acceleration_inputs, sz_inputs, p_throws_inputs, type_inputs], outputs=output)

In [35]:
model.compile(optimizer='adam', loss='mean_squared_error')  # change based on your needs

In [36]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 18)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 14)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 18)]         0                                            
_______________________________________________________________________________________

In [38]:
history = model.fit(
    [X_train_encoded_scaled[['pitch_name_4-seam fastball', 'pitch_name_automatic ball', 'pitch_name_changeup', 'pitch_name_curveball', 'pitch_name_cutter', 'pitch_name_eephus', 'pitch_name_forkball', 'pitch_name_knuckle curve', 'pitch_name_knuckleball', 'pitch_name_other', 'pitch_name_pitch out', 'pitch_name_screwball', 'pitch_name_sinker', 'pitch_name_slider', 'pitch_name_slow curve', 'pitch_name_slurve', 'pitch_name_split-finger', 'pitch_name_sweeper']],
     X_train_encoded_scaled[['events_double', 'events_double_play', 'events_field_out', 'events_force_out', 'events_grounded_into_double_play', 'events_hit_by_pitch', 'events_home_run', 'events_ordinary', 'events_sac_bunt', 'events_sac_fly', 'events_single', 'events_strikeout', 'events_triple', 'events_walk']],
     X_train_encoded_scaled[['description_ball', 'description_blocked_ball', 'description_bunt_foul_tip', 'description_called_strike', 'description_foul', 'description_foul_bunt', 'description_foul_pitchout', 'description_foul_tip', 'description_hit_by_pitch', 'description_hit_into_play', 'description_missed_bunt', 'description_swinging_strike', 'description_swinging_strike_blocked']],
     X_train_encoded_scaled[['pitch_type_ab', 'pitch_type_ch', 'pitch_type_cs', 'pitch_type_cu', 'pitch_type_ep', 'pitch_type_fa', 'pitch_type_fc', 'pitch_type_ff', 'pitch_type_fo', 'pitch_type_fs', 'pitch_type_kc', 'pitch_type_kn', 'pitch_type_po', 'pitch_type_sc', 'pitch_type_si', 'pitch_type_sl', 'pitch_type_st', 'pitch_type_sv']],
     X_train_encoded_scaled[['vx0', 'vy0', 'vz0']],
     X_train_encoded_scaled[['pfx_x', 'pfx_z']],
     X_train_encoded_scaled[['plate_x', 'plate_z']],
     X_train_encoded_scaled[['release_pos_x', 'release_pos_z']],
     X_train_encoded_scaled[['ax', 'ay', 'az']],
     X_train_encoded_scaled[['sz_top', 'sz_bot']],
     X_train_encoded_scaled[['p_throws_l', 'p_throws_r']],
     X_train_encoded_scaled[['type_b', 'type_s', 'type_x']]],
    y_train,
    validation_data=(
        [X_val_encoded_scaled[['pitch_name_4-seam fastball', 'pitch_name_automatic ball', 'pitch_name_changeup', 'pitch_name_curveball', 'pitch_name_cutter', 'pitch_name_eephus', 'pitch_name_forkball', 'pitch_name_knuckle curve', 'pitch_name_knuckleball', 'pitch_name_other', 'pitch_name_pitch out', 'pitch_name_screwball', 'pitch_name_sinker', 'pitch_name_slider', 'pitch_name_slow curve', 'pitch_name_slurve', 'pitch_name_split-finger', 'pitch_name_sweeper']],
         X_val_encoded_scaled[['events_double', 'events_double_play', 'events_field_out', 'events_force_out', 'events_grounded_into_double_play', 'events_hit_by_pitch', 'events_home_run', 'events_ordinary', 'events_sac_bunt', 'events_sac_fly', 'events_single', 'events_strikeout', 'events_triple', 'events_walk']],
         X_val_encoded_scaled[['description_ball', 'description_blocked_ball', 'description_bunt_foul_tip', 'description_called_strike', 'description_foul', 'description_foul_bunt', 'description_foul_pitchout', 'description_foul_tip', 'description_hit_by_pitch', 'description_hit_into_play', 'description_missed_bunt', 'description_swinging_strike', 'description_swinging_strike_blocked']],
         X_val_encoded_scaled[['pitch_type_ab', 'pitch_type_ch', 'pitch_type_cs', 'pitch_type_cu', 'pitch_type_ep', 'pitch_type_fa', 'pitch_type_fc', 'pitch_type_ff', 'pitch_type_fo', 'pitch_type_fs', 'pitch_type_kc', 'pitch_type_kn', 'pitch_type_po', 'pitch_type_sc', 'pitch_type_si', 'pitch_type_sl', 'pitch_type_st', 'pitch_type_sv']],
         X_val_encoded_scaled[['vx0', 'vy0', 'vz0']],
         X_val_encoded_scaled[['pfx_x', 'pfx_z']],
         X_val_encoded_scaled[['plate_x', 'plate_z']],
         X_val_encoded_scaled[['release_pos_x', 'release_pos_z']],
         X_val_encoded_scaled[['ax', 'ay', 'az']],
         X_val_encoded_scaled[['sz_top', 'sz_bot']],
         X_val_encoded_scaled[['p_throws_l', 'p_throws_r']],
         X_val_encoded_scaled[['type_b', 'type_s', 'type_x']]],
        y_val),
    epochs=10)

Epoch 1/10
238609/238609 [==============================] - 212s 887us/step - loss: 10304498688.0000 - val_loss: 8766704640.0000
Epoch 2/10
238609/238609 [==============================] - 208s 872us/step - loss: 9182171136.0000 - val_loss: 13099324416.0000
Epoch 3/10
238609/238609 [==============================] - 210s 881us/step - loss: 8738058240.0000 - val_loss: 14736549888.0000
Epoch 4/10
238609/238609 [==============================] - 210s 880us/step - loss: 8655038464.0000 - val_loss: 14176060416.0000
Epoch 5/10
190424/238609 [======================>.......] - ETA: 42s - loss: 8611676160.0000

KeyboardInterrupt: 

In [41]:
y_val_pred = model.predict([X_val_encoded_scaled[['pitch_name_4-seam fastball', 'pitch_name_automatic ball', 'pitch_name_changeup', 'pitch_name_curveball', 'pitch_name_cutter', 'pitch_name_eephus', 'pitch_name_forkball', 'pitch_name_knuckle curve', 'pitch_name_knuckleball', 'pitch_name_other', 'pitch_name_pitch out', 'pitch_name_screwball', 'pitch_name_sinker', 'pitch_name_slider', 'pitch_name_slow curve', 'pitch_name_slurve', 'pitch_name_split-finger', 'pitch_name_sweeper']],
                             X_val_encoded_scaled[['events_double', 'events_double_play', 'events_field_out', 'events_force_out', 'events_grounded_into_double_play', 'events_hit_by_pitch', 'events_home_run', 'events_ordinary', 'events_sac_bunt', 'events_sac_fly', 'events_single', 'events_strikeout', 'events_triple', 'events_walk']],
                             X_val_encoded_scaled[['description_ball', 'description_blocked_ball', 'description_bunt_foul_tip', 'description_called_strike', 'description_foul', 'description_foul_bunt', 'description_foul_pitchout', 'description_foul_tip', 'description_hit_by_pitch', 'description_hit_into_play', 'description_missed_bunt', 'description_swinging_strike', 'description_swinging_strike_blocked']],
                             X_val_encoded_scaled[['pitch_type_ab', 'pitch_type_ch', 'pitch_type_cs', 'pitch_type_cu', 'pitch_type_ep', 'pitch_type_fa', 'pitch_type_fc', 'pitch_type_ff', 'pitch_type_fo', 'pitch_type_fs', 'pitch_type_kc', 'pitch_type_kn', 'pitch_type_po', 'pitch_type_sc', 'pitch_type_si', 'pitch_type_sl', 'pitch_type_st', 'pitch_type_sv']],
                             X_val_encoded_scaled[['vx0', 'vy0', 'vz0']],
                             X_val_encoded_scaled[['pfx_x', 'pfx_z']],
                             X_val_encoded_scaled[['plate_x', 'plate_z']],
                             X_val_encoded_scaled[['release_pos_x', 'release_pos_z']],
                             X_val_encoded_scaled[['ax', 'ay', 'az']],
                             X_val_encoded_scaled[['sz_top', 'sz_bot']],
                             X_val_encoded_scaled[['p_throws_l', 'p_throws_r']],
                             X_val_encoded_scaled[['type_b', 'type_s', 'type_x']]])

In [42]:
mae = mean_absolute_error(y_val, y_val_pred)
mse = mean_squared_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Absolute Error: 108146.93240414883
Mean Squared Error: 15009565063.359056
R-squared: -0.08131107260004877
